In [ ]:
#file_path="G:/Innovations@HELLER/DN/KI/Temperaturkompensation/Data_Analytics_temp/data_extra_sensors.csv"
'''selected_columns= [ 'T_Bett', 'T_Spindel',
       'DRHZ', 
       'Z_TCP_MK', 'T_Raum_oben', 'T_Raum_unten', 'T_KSS_Tank', 'T_M8',
       'T_M21', 'T_Spindel_Vorl', 'T_Spindel_Rueckl', 'T_Spindelwelle', 'Z_TCP_OK']
       '''
file_path= "G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\row_files\\combines\\smoothed\\combined_file\\combined.csv"
# selected_columns= [
#        't_bett', 't_spindel','smoothed_t_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor', 't_kss_tank', 't_raum',
#        't_spindel_rueck', 't_spindel_vor', 'volumenstrom',
#        'z_schlitten_ok', 'z_hals_ok',
#         'z_spindel_ok', 'z_welle_ok',   
#         'z_tcp_ok_orginal', 'z_tcp_ok']
selected_columns= [
       't_bett', 't_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor', 't_kss_tank', 't_raum',
       't_spindel_rueck', 't_spindel_vor', 'volumenstrom','z_welle_ok']

In [ ]:
import pandas as pd
df= pd.read_csv(file_path)
df= df[selected_columns]
#df['delta_t_spindel'] = df['t_spindel_rueck']-df['t_spindel_vor']
#df['delta_t_spindel_bett'] = df['t_spindel']-df['t_bett']
correlations= df.corr()
df.info()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
fig, ax= plt.subplots(figsize=(20,20))
sn.heatmap(correlations, annot=True, vmin=-1, vmax=1)
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
## Apply PCA
#selected_columns= ['T_Bett', 'T_Spindel','DRHZ','T_Raum_oben', 'T_Raum_unten', 'T_KSS_Tank', 'T_M8','T_M21', 'T_Spindel_Vorl', 'T_Spindel_Rueckl', 'T_Spindelwelle']#  
selected_columns= [
       't_bett', 't_spindel','smoothed_t_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor', 't_kss_tank', 't_raum',
       't_spindel_rueck', 't_spindel_vor', 'volumenstrom']    
X= df[selected_columns]
y= df['z_tcp_ok']
x_train, x_test, y_train, y_test= train_test_split(X, y, shuffle=True, test_size= 0.1)
stdScaler= MinMaxScaler(feature_range=(-1,1))#StandardScaler(with_mean= True, with_std= True)
x_train= stdScaler.fit_transform(x_train)
x_test= stdScaler.transform(x_test)
pca= PCA(5)
x_train_pca= pca.fit_transform(x_train)
x_test_pca= pca.transform(x_test)
print ('Explainability of Components: ', pca.explained_variance_ratio_)

In [ ]:
### Train a model and test it
###Model of KNN
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
knn= KNeighborsRegressor(n_neighbors= 5, weights='distance')
#pipline2= Pipeline(steps=[('knn', knn)])
results= cross_validate(estimator= knn, X= x_train_pca, y= y_train,cv= 5,scoring= ( 'r2','neg_mean_squared_error','neg_mean_absolute_error','neg_mean_absolute_percentage_error'), verbose= 2,n_jobs=8)
print ('mae= ', results['test_neg_mean_absolute_error'].mean(), ' mape= ', results['test_neg_mean_absolute_percentage_error'].mean(),' mse= ', results['test_neg_mean_squared_error'].mean(), ' r2= ', results['test_r2'].mean() )


In [ ]:
### Create Tolerance Lines
import numpy as np
tolerance= 0.005
x_steps= np.linspace(-0.03, 0.05, 10)
lowerline= x_steps - tolerance
upperline= x_steps + tolerance

In [ ]:
###Test KNN
knn= KNeighborsRegressor(n_neighbors= 5, weights='distance')
knn= knn.fit(X= x_train_pca, y= y_train)
y_pred= knn.predict(x_test_pca)
fig, ax= plt.subplots(figsize=(15,15))
plt.title('Results of KNN Algorithm')
plt.grid()
plt.xlabel('True')
plt.ylabel('Predicted')
plt.plot(x_steps, upperline, label= 'Upper Bound')
plt.scatter(y_test,y_pred, label= 'predictions')
plt.plot(x_steps, lowerline, label= 'lower Bound')
plt.plot(x_steps, x_steps, label= 'Optimal line')
plt.legend()
plt.show()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
print ('r2 ', r2_score(y_test,y_pred),' mse= ',mean_squared_error(y_test,y_pred), ' mae= ', mean_absolute_error(y_test,y_pred), ' mape= ', mean_absolute_percentage_error(y_test,y_pred))

In [ ]:
###Model of Random Forest
from sklearn.ensemble import RandomForestRegressor
randomForest= RandomForestRegressor(criterion='mae',random_state= 47, bootstrap= True)
results= cross_validate(estimator= randomForest, X=x_train, y= y_train,cv= 50,scoring= ( 'neg_mean_squared_error','neg_mean_absolute_error','neg_mean_absolute_percentage_error', 'r2'), verbose= 2,n_jobs=8)
print ('r2= ', results['test_r2'].mean(),' mae= ', results['test_neg_mean_absolute_error'].mean(), ' mape= ', results['test_neg_mean_absolute_percentage_error'].mean(),' mse= ', results['test_neg_mean_squared_error'].mean() )

In [ ]:
##Train and Test Random Forest
randomForest= RandomForestRegressor(criterion='mae',random_state= 47, bootstrap= True)
randomForest=randomForest.fit(x_train, y=y_train)
y_pred= randomForest.predict(x_test)
fig, ax= plt.subplots(figsize=(15,15))
plt.title('Results of RandomForest Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
plt.plot(x_steps, upperline, label= 'Upper Bound')
plt.scatter(y_test,y_pred, label= 'predictions')
plt.plot(x_steps, lowerline, label= 'lower Bound')
plt.plot(x_steps, x_steps, label= 'Optimal line')
plt.legend()
plt.show()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
print ('mse= ',mean_squared_error(y_test,y_pred), ' mae= ', mean_absolute_error(y_test,y_pred), ' mape= ', mean_absolute_percentage_error(y_test,y_pred))

Apply Analysis with always available data

In [ ]:
###Analysis with T_Spindel
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
## Apply PCA
selected_columns= ['T_Bett', 'T_Spindel','DRHZ']
X= df[selected_columns]
y= df['Z_TCP_OK']
x_train, x_test, y_train, y_test= train_test_split(X, y, shuffle=True, test_size= 0.1)
stdScaler= MinMaxScaler(feature_range=(-1,1))#StandardScaler(with_mean= True, with_std= True)
x_train= stdScaler.fit_transform(x_train)
x_test= stdScaler.transform(x_test)
pca= PCA(2)
x_train_pca= pca.fit_transform(x_train)
x_test_pca= pca.transform(x_test)
print ('Explainability of Components: ', pca.explained_variance_ratio_)

In [ ]:
### Train a model and test it
###Model of KNN
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_validate
knn= KNeighborsRegressor(n_neighbors= 5, weights='distance')
#pipline2= Pipeline(steps=[('knn', knn)])
results= cross_validate(estimator= knn, X= x_train_pca, y= y_train,cv= 5,scoring= ( 'r2','neg_mean_squared_error','neg_mean_absolute_error','neg_mean_absolute_percentage_error'), verbose= 2,n_jobs=8)
print ('mae= ', results['test_neg_mean_absolute_error'].mean(), ' mape= ', results['test_neg_mean_absolute_percentage_error'].mean(),' mse= ', results['test_neg_mean_squared_error'].mean(), ' r2= ', results['test_r2'].mean() )


In [ ]:
###Test KNN
knn= KNeighborsRegressor(n_neighbors= 5, weights='distance')
knn= knn.fit(X= x_train_pca, y= y_train)
y_pred= knn.predict(x_test_pca)
fig, ax= plt.subplots(figsize=(15,15))
plt.grid()
plt.title('Results of KNN Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.plot(x_steps, upperline, label= 'Upper Bound')
plt.scatter(y_test,y_pred, label= 'predictions')
plt.plot(x_steps, lowerline, label= 'lower Bound')
plt.plot(x_steps, x_steps, label= 'Optimal line')
plt.legend()
plt.show()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
print ('r2 ', r2_score(y_test,y_pred),' mse= ',mean_squared_error(y_test,y_pred), ' mae= ', mean_absolute_error(y_test,y_pred), ' mape= ', mean_absolute_percentage_error(y_test,y_pred))

In [ ]:
###Model of Random Forest
from sklearn.ensemble import RandomForestRegressor
randomForest= RandomForestRegressor(criterion='mae',random_state= 47, bootstrap= True)
results= cross_validate(estimator= randomForest, X=x_train, y= y_train,cv= 50,scoring= ( 'neg_mean_squared_error','neg_mean_absolute_error','neg_mean_absolute_percentage_error', 'r2'), verbose= 2,n_jobs=8)
print ('r2= ', results['test_r2'].mean(),' mae= ', results['test_neg_mean_absolute_error'].mean(), ' mape= ', results['test_neg_mean_absolute_percentage_error'].mean(),' mse= ', results['test_neg_mean_squared_error'].mean() )

In [ ]:
##Train and Test Random Forest
randomForest= RandomForestRegressor(criterion='mae',random_state= 47, bootstrap= True)
randomForest=randomForest.fit(x_train, y=y_train)
y_pred= randomForest.predict(x_test)
fig, ax= plt.subplots(figsize=(15,15))
plt.title('Results of Random Forest Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
plt.plot(x_steps, upperline, label= 'Upper Bound')
plt.scatter(y_test,y_pred, label= 'predictions')
plt.plot(x_steps, lowerline, label= 'lower Bound')
plt.plot(x_steps, x_steps, label= 'Optimal line')
plt.legend()
plt.show()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
print ('mse= ',mean_squared_error(y_test,y_pred), ' mae= ', mean_absolute_error(y_test,y_pred), ' mape= ', mean_absolute_percentage_error(y_test,y_pred))

In [ ]:
df1= pd.read_csv(file_path, sep=';', decimal= '.')
selected_columns= [ 'DAT_UHZ', 'T_Bett', 'T_Spindel',
       'DRHZ',  'T_Raum_oben', 'T_Raum_unten', 'T_KSS_Tank', 'T_M8',
       'T_M21', 'T_Spindel_Vorl', 'T_Spindel_Rueckl', 'T_Spindelwelle', 'Z_TCP_OK']
df= df1[selected_columns]
df= df[df['T_Spindelwelle'] == df['T_Spindelwelle']]
df.info()
#### Generate T_Spindelwelle artificially
selected_columns= ['T_Bett', 'T_Spindel','DRHZ']
X= df[selected_columns]
y= df['T_Spindelwelle']
x_train, x_test, y_train, y_test= train_test_split(X, y, shuffle=True, test_size= 0.1)

In [ ]:
pca= PCA(1)
x_train_pca= pca.fit_transform(x_train)
x_test_pca= pca.transform(x_test)
print ('Explainability of Components: ', pca.explained_variance_ratio_)


In [ ]:
knn= KNeighborsRegressor(n_neighbors= 5, weights='distance')
results= cross_validate(estimator= knn, X= x_train_pca, y= y_train,cv= 5,scoring= ( 'r2','neg_mean_squared_error','neg_mean_absolute_error','neg_mean_absolute_percentage_error'), verbose= 2,n_jobs=8)
print ('mae= ', results['test_neg_mean_absolute_error'].mean(), ' mape= ', results['test_neg_mean_absolute_percentage_error'].mean(),' mse= ', results['test_neg_mean_squared_error'].mean(), ' r2= ', results['test_r2'].mean() )

In [ ]:
###Test KNN
knn= KNeighborsRegressor(n_neighbors= 5, weights='distance')
knn= knn.fit(X= x_train_pca, y= y_train)
y_pred= knn.predict(x_test_pca)
fig, ax= plt.subplots(figsize=(15,15))
plt.grid()
plt.title('Results of KNN Algorithm')
plt.xlabel('T_Spindelwelle_True')
plt.ylabel('T_Spindelwelle_Predicted')
plt.scatter(y_test,y_pred, label= 'predictions')
plt.legend()
plt.show()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score
print ('r2 ', r2_score(y_test,y_pred),' mse= ',mean_squared_error(y_test,y_pred), ' mae= ', mean_absolute_error(y_test,y_pred), ' mape= ', mean_absolute_percentage_error(y_test,y_pred))

In [ ]:
X_pca= pca.transform(X)
df['predicted_T_Spindelwelle']= knn.predict(X_pca)
fig, ax= plt.subplots(figsize=(15,15))
plt.grid()
plt.title('Results of KNN Algorithm on T_Spindelwelle')
plt.xlabel('T_Spindelwelle_True')
plt.ylabel('T_Spindelwelle_Predicted')
plt.plot(df['T_Spindelwelle'], label='T_Spindelwelle_Orginal')
plt.plot(df['predicted_T_Spindelwelle'], label='T_Spindelwelle_ Predicted')
plt.legend()
plt.show

In [ ]:
## Generate 'T_Spindelwelle' for all data
columns= ['T_Bett', 'T_Spindel','DRHZ']
X= df1[columns]
y= df1['T_Spindelwelle']
X_pca= pca.transform(X)
df1['predicted_T_Spindelwelle']= knn.predict(X_pca)
fig, ax= plt.subplots(figsize=(15,15))
plt.grid()
plt.title('Results of KNN Algorithm on T_Spindelwelle')
plt.xlabel('Time')
plt.ylabel('T_Spindelwelle_Predicted')
#plt.plot(df['T_Spindelwelle'], label='T_Spindelwelle_Orginal')
plt.plot(df1['predicted_T_Spindelwelle'], label='T_Spindelwelle_ Predicted')
plt.legend()
plt.show()

In [ ]:
## Train a Random Forest Model based on new data
###Model of Random Forest
selected_columns= ['T_Bett', 'T_Spindel','DRHZ',  'T_Raum_oben', 'T_Raum_unten', 'T_KSS_Tank', 'T_M8',
       'T_M21', 'T_Spindel_Vorl', 'T_Spindel_Rueckl', 'predicted_T_Spindelwelle']
X= df1[selected_columns]
y= df1['Z_TCP_OK']
x_train, x_test, y_train, y_test= train_test_split(X, y, shuffle=True, test_size= 0.1)
from sklearn.ensemble import RandomForestRegressor
randomForest= RandomForestRegressor(criterion='mae',random_state= 47, bootstrap= True)
results= cross_validate(estimator= randomForest, X=x_train, y= y_train,cv= 50,scoring= ( 'neg_mean_squared_error','neg_mean_absolute_error','neg_mean_absolute_percentage_error', 'r2'), verbose= 2,n_jobs=8)
print ('r2= ', results['test_r2'].mean(),' mae= ', results['test_neg_mean_absolute_error'].mean(), ' mape= ', results['test_neg_mean_absolute_percentage_error'].mean(),' mse= ', results['test_neg_mean_squared_error'].mean() )

In [ ]:
##Train and Test Random Forest
randomForest= RandomForestRegressor(criterion='mae',random_state= 47, bootstrap= True)
randomForest=randomForest.fit(x_train, y=y_train)
y_pred= randomForest.predict(x_test)
fig, ax= plt.subplots(figsize=(15,15))
plt.title('Results of Random Forest Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
plt.plot(x_steps, upperline, label= 'Upper Bound')
plt.scatter(y_test,y_pred, label= 'predictions')
plt.plot(x_steps, lowerline, label= 'lower Bound')
plt.plot(x_steps, x_steps, label= 'Optimal line')
plt.legend()
plt.show()
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
print ('mse= ',mean_squared_error(y_test,y_pred), ' mae= ', mean_absolute_error(y_test,y_pred), ' mape= ', mean_absolute_percentage_error(y_test,y_pred))